In [0]:
%run /Users/pnl0rc8b@emea.royalahold.net/FINALIZED_Notebooks/Functions_Finalized

In [0]:
%run /Users/pnl0rc8b@emea.royalahold.net/FINALIZED_Notebooks/Functions_Tables

#Configureer Parameters

In [0]:
ATTRIBUUT = "ndDairyType" # Selecteer Attribuut

#Vul dezelfde waardes in als tijdens trainen:
Classes_threshold = 11
Overig_threshold = 50 # Class beneden deze threshold, zal worden toegevoegd aan class: 'overig'

In [0]:
numerieke_features = load_numerical_features(ATTRIBUUT)

Numerieke Features: []

# Fix empty rows

In [0]:
nd_controle = koppel_ndAttributen_new_products(NASA_CONTROLE)
df_geformateerd, multi_label_flag,classes_available = create_overige_class(nd_controle, ATTRIBUUT,Classes_threshold, Overig_threshold)

DF_merged = koppel_preprocessed_text(df_geformateerd, DF_NLP_spark)
DF_merged = fix_empty_fields(DF_merged, ATTRIBUUT)
DF_merged = Correlation_matrix(DF_merged, ATTRIBUUT, show_nan_plot=False, show_correlation_matrix=False)

index_empty, TARGETS = get_targets(DF_merged, ATTRIBUUT)
numerical, text, target_unused, vocab_size_unused = single_prep_controle(numerieke_features, DF_merged, ATTRIBUUT)

Amount of rows: 34439 Amount of columns: 139
ATTRIBUUT: ndDairyType
Kleine classes die onder overige moeten: ['Cacaoboter', 'Chocomelk', 'Creme fraiche', 'Halvarine', 'Karnemelk', 'Kefir', 'Koffiemelk', 'Kruidenboter', 'Margarine', 'Pudding', 'Skyr', 'Vla', 'Zuivelspread', 'Zure room']
Kleine classes, aantallen: [26, 42, 4, 7, 17, 11, 20, 16, 42, 36, 29, 19, 8, 16]
Grotere classes: ['Boter', 'Drinkyoghurt', 'EMPTY', 'Kwark', 'Melk', 'Room', 'Roomboter', 'Slagroom', 'Yoghurt']
Grotere classes, aantallen: [57, 58, 33145, 83, 245, 293, 199, 53, 328]
null entries = 0
Shape van dataframe: (23162, 34)
null entries = 683
Targets: ['Boter', 'Drinkyoghurt', 'EMPTY', 'Kwark', 'Melk', 'Room', 'Roomboter', 'Slagroom', 'Yoghurt', 'overig']
Index of Empty: 2
Vocabular size: 39801.0
Numerical Features: Index([], dtype='object')
Shape Numerical Features: (22479, 0)
Shape Text Features: (22479, 301)
Shape Targets: (22479, 10, 2)

# Laad modellen van database in, (als voorbeeld: ndTypeOfGrain) bestaat uit meerdere modellen.

In [0]:
from tensorflow import keras
from tqdm import tqdm
all_model_files = dbutils.fs.ls("dbfs:/FileStore/nd_MODELS/"+str(ATTRIBUUT))
#print(all_model_files)
controle_tabel = DF_merged[['gtin','gln','Lemmatized',ATTRIBUUT]]
total_pred = []
total_test = []
class_names = []

for index, model in enumerate(tqdm(all_model_files)): # loop through all models
    model_path = model.path
    dbutils.fs.cp(model.path, "file:/FileStore/nd_MODELS/"+str(ATTRIBUUT)+"/"+model.name)
    model = keras.models.load_model("/FileStore/nd_MODELS/"+str(ATTRIBUUT)+"/"+model.name)

    # TARGET
    target = TARGETS[index]
    print('\n\n', target, ' ----> ',model_path)
    
    y_target = DF_merged[str(target)].tolist()
    y_target_tensor = torch.FloatTensor(y_target)
    y_target = y_target_tensor.numpy()
    print(numerical.shape, text.shape, y_target.shape)

    # Predictions
    list_ytest, list_yhat, yhat2 = predictions_2inputs(model, numerical, text, y_target)
    
    controle_tabel['TRUE '+str(target)] = list_ytest
    controle_tabel['PRED '+str(target)] = list_yhat
    total_test.append(list_ytest)
    total_pred.append(list_yhat)
    class_names.append(target)

0%| | 0/10 [00:00<?, ?it/s]

 Boter ----> dbfs:/FileStore/nd_MODELS/ndDairyType/model_Boter.h5
(22479, 0) (22479, 301) (22479, 2)
 1/703 [..............................] - ETA: 1:34 41/703 [>.............................] - ETA: 0s  87/703 [==>...........................] - ETA: 0s136/703 [====>.........................] - ETA: 0s183/703 [======>.......................] - ETA: 0s229/703 [========>.....................] - ETA: 0s276/703 [==========>...................] - ETA: 0s323/703 [============>.................] - ETA: 0s371/703 [==============>...............] - ETA: 0s419/703 [================>.............] - ETA: 0s467/703 [==================>...........] - ETA: 0s515/703 [====================>.........] - ETA: 0s565/703 [=======================>......] - ETA: 0s615/703 [=========================>....] - ETA: 0s665/703 [===========================>..] - ETA: 0s703/703 [==============================] - 1s 1ms/step
 1/703 [..............................] - ETA: 1:35 - loss: 2.3002e-04 - accuracy: 1.0000 42/703 [>.............................] - ETA: 0s - loss: 2.0513e-04 - accuracy: 1.0000  85/703 [==>...........................] - ETA: 0s - loss: 3.1583e-04 - accuracy: 1.0000128/703 [====>.........................] - ETA: 0s - loss: 2.9275e-04 - accuracy: 1.0000172/703 [======>.......................] - ETA: 0s - loss: 8.7679e-04 - accuracy: 0.9998213/703 [========>.....................] - ETA: 0s - loss: 0.0028 - accuracy: 0.9996 257/703 [=========>....................] - ETA: 0s - loss: 0.0029 - accuracy: 0.9995301/703 [===========>..................] - ETA: 0s - loss: 0.0037 - accuracy: 0.9994345/703 [=============>................] - ETA: 0s - loss: 0.0052 - accuracy: 0.9993389/703 [===============>..............] - ETA: 0s - loss: 0.0047 - accuracy: 0.9994432/703 [=================>............] - ETA: 0s - loss: 0.0042 - accuracy: 0.9994476/703 [===================>..........] - ETA: 0s - loss: 0.0048 - accuracy: 0.9993521/703 [=====================>........] - ETA: 0s - loss: 0.0049 - accuracy: 0.9993565/703 [=======================>......] - ETA: 0s -

# Controle Tabellen

In [0]:
controle_tabel = predictions_to_right_format(total_test,total_pred, ATTRIBUUT, index_empty, controle_tabel, DF_merged)
equal, unequal = unequal_and_equal_table(controle_tabel, ATTRIBUUT) # Both spark dataframes

empty dummy [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
complete empty dummy [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [0]:
display(unequal)# Met deze tabel, kan je inzien welke producten er nog moeten worden gecontroleerd. Dus hier is busines-rules ongelijk aan de voorspellingen van het machine learning model.

gtin,gln,NASA_NR,MERKNAAM,SCHAPSTICKER_OMSCHRIJVING,ASSGROEP_OMSCHRIJVING,Lemmatized,ndDairyType,PREDICTED Labels,UnLemmatized
5410146406676,5410146000003,657130,Danone,danup banaan bel,B zuivel,drinkyoghurt banaan drinkyoghurt vol danone danup banaan 600 g danup 600 banaan ingrediënt afromen melk suiker 96 banaan 23 waarvan 01 concentreren sap room melk aroma zuurteregelaars citroenzuur ascorbinezuur yoghurtferment melk kleurstof luteïne verdikkingsmiddel johannesbroodpitmeel,Drinkyoghurt,List(EMPTY),drinkyoghurt banaan drinkyoghurt vol danone danup banaan 600 g danup 600 banaan ingrediënten afgeroomde melk suiker 96 banaan 23 waarvan 01 geconcentreerd sap room melk aroma zuurteregelaars citroenzuur ascorbinezuur yoghurtfermenten melk kleurstof luteïne verdikkingsmiddel johannesbroodpitmeel
8714200218456,8713800999994,747923,Venco,venco chocod mlk an,Suikerwerk,melkchocolade dropvulling bestrooien droppoed anijssmaak melkchocolade dropvulling venco chocodrop melk anijs 146 g venco chocodrop melk anijs 146 g ingrediënt melkchocolade 77 suiker vol melkpoeder cacaomassa1 cacaoboter1 weipoeder melk emulgator sojalecithin natuurlijk vanill aroma drop20 melado tarwebloem suiker zoethoutwortelextract salmiakzout emulgator e471 kokosolie anijs aroma kokosvet droppoeder anijssmaak 2 suiker zoethoutwortelextract kleurstof e153 salmiakzout aroma glucosestroop glansmiddel arabisch gom 1 rainforest alliance certified ™ find out more at raorg noot pinda bevatten,Melk,List(EMPTY),melkchocolade dropvulling bestrooid droppoeder anijssmaak melkchocolade dropvulling venco chocodrop melk anijs 146g venco chocodrop melk anijs 146g ingrediënten melkchocolade 77 suiker volle melkpoeder cacaomassa1 cacaoboter1 weipoeder melk emulgator sojalecithine natuurlijk vanille aroma drop20 melado tarwebloem suiker zoethoutwortelextract salmiakzout emulgator e471 kokosolie anijs aroma kokosvet droppoeder anijssmaak 2 suiker zoethoutwortelextract kleurstof e153 salmiakzout aroma glucosestroop glansmiddel arabische gom 1 rainforest alliance certified™ find out more at raorg noten pindas bevatten
8719200228030,8719200069381,1855,Blue Band,blue band halvarine,Boter & Margarine,halvarin product 39 plantaardig olie halvarine blue band halvarine 500 g blue band halvarine 500 g bekerkuipje ingrediëntenwater plantaardig olie 39zonnebloem 14 palm koolzaad 77 lijnzaad 45 modificeren zetmeel zout 035 emulgator mono diglycerid vetzuur zonnebloemlecithine voedingszuur citroenzuur natuurlijk aroma vitamine a b1 b2 b6 foliumzuur b12 d,Halvarine,List(EMPTY),halvarine product 39 plantaardige oliën halvarine blue band halvarine 500 g blue band halvarine 500 g bekerkuipje ingrediëntenwater plantaardige oliën 39zonnebloem 14 palm koolzaad 77 lijnzaad 45 gemodificeerd zetmeel zout 035 emulgatoren mono diglyceriden vetzuren zonnebloemlecithine voedingszuur citroenzuur natuurlijke aromas vitamine a b1 b2 b6 foliumzuur b12 d
3270160827398,8711145000016,738578,Picard,picard 4 eclairs au choc,Picard,eclairs chocolade soezendeeg banketbakkersroom chocolade chocoladeglazuur diepvriezen gebak minieclair picard eclairs choc picard eclairs choc 240 g ingrediënt gerehydrateeren mager melk chocolade 14 cacaomassa suiker cacaoboter emulgator sojalecithin ei vloeibaar pasteuriseren water halfvol melk tarwemeel glut suiker boterconcentraat melk maïszetmeel melkchocolade suiker halfvol melk poeder cacaoboter cacaomassa mager melk poeder emulgator sojalecithin magerecacaopoeder invertsuikerstroop inuline zout geleermiddel carrage product verwerken bedrijf waar volgen zaak gebruiken aardnoot nott,EMPTY,List(Melk),eclairs chocolade soezendeeg banketbakkersroom chocolade chocoladeglazuur diepgevroren gebak minieclairs picard eclairs choc picard eclairs choc 240g ingrediënten gerehydrateerde magere melk chocolade 14 cacaomassa suiker cacaoboter emulgator sojalecithine ei vloeibaar gepasteuriseerd water halfvolle melk tarwemeel gluten suiker boterconcentraat melk maïszetmeel melkchocolade suiker halfvol

In [0]:
display(equal)

gtin,gln,NASA_NR,MERKNAAM,SCHAPSTICKER_OMSCHRIJVING,ASSGROEP_OMSCHRIJVING,Lemmatized,ndDairyType,PREDICTED Labels,UnLemmatized
7622210998477,8711872000174,844207,LU,lu tuc sweet chili,Noten & Toast,cracker zacht chilipepersmaak spice lu tuc cracker sweet chili smaak 100 g lu tuc spice sweet chili ingrediënt tarwebloem zonnebloemolie 22 glucosestroop rijsmiddel ammoniumcarbonat natriumcarbonaten zout suiker aromas dextrose weipoeder melk zuurteregelaar natriumacetat voedingszuur citroenzuur rood chilipeper 004 meelverbeteraar natriumdisulfiet ei soja sesam bevatten,EMPTY,List(EMPTY),crackers zachte chilipepersmaak spices lu tuc crackers sweet chili smaak 100 g lu tuc spices sweet chili ingrediënten tarwebloem zonnebloemolie 22 glucosestroop rijsmiddelen ammoniumcarbonaten natriumcarbonaten zout suiker aromas dextrose weipoeder melk zuurteregelaar natriumacetaten voedingszuur citroenzuur rode chilipepers 004 meelverbeteraar natriumdisulfiet ei soja sesam bevatten
8710565007476,8710565000095,845168,Bakkernij,bakk rb kan krakelingen,Koek,kaneel krakeling koeken bakkernij krakeling roombot kaneel 240gr bakkernij krakeling roombot kaneel 240gr ingrediënt tarwebloem 26 roomboter melk suiker gist zout kaneel,Roomboter,"List(Room, Roomboter)",kaneel krakelingen koeken bakkernij krakelingen roomboter kaneel 240gr bakkernij krakelingen roomboter kaneel 240gr ingrediënten tarwebloem 26 roomboter melk suiker gist zout kaneel
8710912249467,8710912888888,693063,Milner,milner jong bel komijn pl,Kaas ZB,komijn kaas 30 maken halfvol melk kaas milner komijn kaas 30 plakken 175 g milner kaas komijn 30 plakken 175 g tray ingrediënt halfvol melk zout komijn zuursel vegetarisch stremsel conserveermiddel e251 kleurstof bètacaroteen,EMPTY,List(EMPTY),komijn kaas 30 gemaakt halfvolle melk kaas milner komijn kaas 30 plakken 175 g milner kaas komijn 30 plakken 175 g tray ingrediënten halfvolle melk zout komijn zuursel vegetarisch stremsel conserveermiddel e251 kleurstof bètacaroteen
8711544124429,8711544000006,747564,Dixap,dixap aar b 12+ bio,Babyvoeding,concentreren sap verscheiden vrucht groente fruit veggie biologisch covelt dixap fruit veggie biologisch appel aardbei biet 400 milliliter covelt dixap fruit veggie biologisch appel aardbei biet 400 milliliter ingrediënt concentreren sap appel 756 biet 20 aardbei 44 biologisch oorsprong suiker toevoegen bevatten alleen natuur aanwezig suiker,EMPTY,List(EMPTY),geconcentreerd sap verscheidene vruchten groenten fruit veggie biologisch covelt dixap fruit veggie biologisch appel aardbei biet 400 milliliter covelt dixap fruit veggie biologisch appel aardbei biet 400 milliliter ingrediënten geconcentreerd sap appel 756 biet 20 aardbei 44 biologische oorsprong suikers toegevoegd bevat alleen nature aanwezige suikers
5410228150404,8725000000011,562833,Jupiler,jupiler bier 6bl,Bier Pils,bier pilsener jupiler belgisch pils blikken 6 x 33 cl jupiler pilsener blik ingrediënt water gerstemout maïs hop,EMPTY,List(EMPTY),bier pilseners jupiler belgische pils blikken 6 x 33 cl jupiler pilsener blik ingrediënten water gerstemout maïs hop
3760278204148,1094207999990,716567,Sushi Daily,rice chilli cracker,Sushi bediening artikelen,rice chilli cracker,EMPTY,List(EMPTY),rice chilli crackers
5390691017919,5425017540007,842442,FulFil,fulfil pean car bar,Functional Foods,eiwitrijk reep melkchocoladelaag pinda karamelsmaak toevoegen vitamine zoetstof proteïnebar fulfil chocolate peanut caramel vitamin protein bar 55 g fulfil proteinebar pindanoot karamel nl ingrediënt melkchocolade zoetstof 282 zoetstof maltitol cacaoboter vol melkpoeder cacaomassa emulgator sojalecithinen aroma melkeiwit laag karamelsmaak 127 vulstof polydextros sojaolie collageenhydrolysaat mager melkpoeder stabilisator xylitol melkeiwit emulgator sojalecithin bevochtigingsmiddel glycerol aroma zout zoetstof sucralose collageenhydrolysaat bevochtigingsmiddel glycerol pinda 55 sojakrokant sojaeiwit zoetstof erythritol sucralos maïsvezel aroma bevatten pindas zou